In [1]:
import sys
#sys.path.append("/Users/manuelladron/iCloud_archive/Documents/_CMU/PHD-CD/PHD-CD_Research/multilabel_classifier/py_files")
#sys.path.append("/Users/manuelladron/phd_cd/DL_11785/homework/hw3/hw3p1/mytorch/")

In [2]:
import torch
import torch.utils.data as data
import os
import time
import pickle
import numpy as np
from PIL import Image

import json
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F


import torch.nn as nn
import torch.optim as optim

import itertools
import collections
import pdb
cuda = torch.cuda.is_available()
cuda

False

#### Fetch Data

In [10]:
# IMAGES
im_path_fur = "../ADARI/images/furniture/thumbs/big"

# JSON_FILES
data_path_fur = "../ADARI/json_files/cleaned/furniture_cleaned.json"

# FURNITURE VOCAB 
vocab_id2w = "../ADARI/json_files/vocabulary/furniture/vocab_id2w.json"
vocab_w2id = "../ADARI/json_files/vocabulary/furniture/vocab_w2id.json"

# WORD EMBEDDINGS
word_embeddings_path = "../ADARI/word_embeddings/fur_5c_50d_sk_glove_ft.json"

# FILES FOR DATALOADER
dset_path = '../ADARI/json_files/dataset_for_dataloader/dset_dataloader.json'
im2idx_path = '../ADARI/json_files/dataset_for_dataloader/im2idx.json'




### Load files

In [11]:
# Path for file dset_dataloader.json
def open_json(path):
    f = open(path) 
    data = json.load(f) 
    f.close()
    return data 

#### Word embeddings 

In [12]:
word_embs = open_json(word_embeddings_path)

#### Vocab 

In [13]:
vocab = open_json(vocab_w2id)

#### Dataset_dataloader

In [14]:
dset = open_json(dset_path)
im2idx = open_json(im2idx_path)

In [15]:
len(dset)

10825

In [16]:
len(dset.keys())

10825

In [17]:
img_size = 64
class ImageDataset(Dataset):
    def __init__(self, dataset, im2idx, path_to_images, train=True):
        self.img_path = path_to_images
        self.data = dataset
        self.im2idx = im2idx
        self.images = list(dataset.keys())
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image_name = self.images[index]
        idx = self.im2idx[image_name]
        
        name = self.img_path + "/" + image_name
        img = Image.open(name)
        
        img = transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
        transforms.ToTensor()])(img)
        
        return img, idx
                
def collate(sequence):
    """
    "the input of this function is the output of function __getitem__"
    "this gets BATCH_SIZE times GETITEM! "
    if batch_Size == 2 --> sequence is a list with length 2. 
    Each list is a tuple (image, label) = ((3,64,64), label_length)
    """

    # Concatenate all images in the batch
    images = torch.cat(([batch_[0].view(-1, 3, 64, 64) for batch_ in sequence]), dim=0)
    
    # Pad labels with max_sequence_label
    idxs = torch.LongTensor([batch_[1] for batch_ in sequence])     
    
    return images, idxs


In [18]:
dataset = ImageDataset(dset, im2idx, im_path_fur, train=True)

In [19]:
class EncoderCNN(nn.Module):
    def __init__(self):
        """Load the pretrained ResNet-152 and replace top fc layer."""
        super(EncoderCNN, self).__init__()
        resnet = models.resnet152(pretrained=True)
        modules = list(resnet.children())[:-1]      # delete the last fc (classification) layer.
        self.resnet = nn.Sequential(*modules)
        
    def forward(self, images):
        """Extract feature vectors from input images."""
        with torch.no_grad():
            features = self.resnet(images)
        return features

In [20]:
img_embedder = EncoderCNN()

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /Users/alexschneidman/.cache/torch/checkpoints/resnet152-b121ed2d.pth


HBox(children=(FloatProgress(value=0.0, max=241530880.0), HTML(value='')))

In [22]:
batch_size = 1
num_workers = 8 if cuda else 0
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn = collate, shuffle=False, num_workers=num_workers, drop_last=False)


### Run this cell to get image embeddings from ResNet152 

In [25]:
image_embeddings = dict() # dictionary to store image embeddings
with torch.no_grad():
    for i, (images, idx) in enumerate(dataloader):

        batch_size = images.shape[0]
        images = images.to(device)
        idx = idx.to(device)   
        # Encode image with CNN
        features = img_embedder(images).squeeze(3).squeeze(2) # shape. [batch, 2048]
        
        # Dictonary key:image_idx, embedding
        image_embeddings[idx.item()] = features

        

In [32]:
reshape_embeddings = dict()
for k, v in image_embeddings.items():
    reshape_embeddings[k] = v.reshape(v.shape[1]).tolist()
with open("resnet_image_embeddings.json", "w") as f:
    json.dump(reshape_embeddings, f)